# Predicciones Meteorológicas (AEMET) - SPRINT I

Parte 1 - Extracción de Datos

Navegar la documentación de la API de AEMET y explorar los endpoints

Desarrollar un script que extraiga la información histórica de todas las provincias.

Ejecutar el script para extraer los datos de los últimos dos años y verificar que todo funcione correctamente.

En el modelo de datos, cada registro debe tener un timestamp de extracción y un identificador para que se pueda manejar el sistema de actualización.

In [ ]:
import os
import requests
import time
import pandas as pd
from datetime import datetime, timedelta
from dotenv import load_dotenv
import uuid

load_dotenv()
API_KEY = os.getenv("AEMET_API_KEY")
if not API_KEY:
    raise RuntimeError("No encontré la clave AEMET_API_KEY en el entorno.")

def obtener_inventario():
    """
    Bajamos la lista completa de estaciones
    """
    url = (
        "https://opendata.aemet.es/opendata/api/"
        "valores/climatologicos/inventarioestaciones/todasestaciones"
    )
    respuesta = requests.get(url, params={"api_key": API_KEY}, timeout=15)
    respuesta.raise_for_status()
    enlace = respuesta.json().get("datos")
    if not enlace:
        raise RuntimeError("No salió la URL de datos del inventario.")
    r2 = requests.get(enlace, timeout=15)
    r2.raise_for_status()
    estaciones = r2.json()
    return pd.DataFrame(estaciones)

def descargar_estacion(indicativo, nombre, id_descarga, intervalos):
    """
    Para una estación, bajamos los datos de cada intervalo de fechas
    """
    registros = []
    for inicio, fin in intervalos:
        url_meta = (
            "https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/"
            f"datos/fechaini/{inicio}/fechafin/{fin}/estacion/{indicativo}"
        )
        try:
            r = requests.get(url_meta, params={"api_key": API_KEY}, timeout=15)
            if r.status_code != 200:
                continue
            meta = r.json()
            enlace_real = meta.get("datos")
            if not enlace_real:
                continue

            r2 = requests.get(enlace_real, timeout=15)
            if r2.status_code != 200:
                continue

            datos_json = r2.json()
            for item in datos_json:
                item["idema"] = indicativo
                item["nombre_estacion"] = nombre
                item["timestamp_extraccion"] = datetime.utcnow().isoformat()
                item["id_descarga"] = id_descarga
                registros.append(item)

            time.sleep(1.2)
        except requests.RequestException:
            # Si falla, seguimos con el siguiente intervalo
            continue
    return registros

def main():
    print("Empezamos a bajar datos de todas las estaciones...")

    # 1) Primero obtengo lista de las estaciones
    estaciones_df = obtener_inventario().dropna(subset=["indicativo"])

    # 2) Creamos intervalos de 6 meses para los últimos 2 años
    hoy = datetime.utcnow().date()
    hace_dos = hoy - timedelta(days=730)
    intervalos = []
    inicio = hace_dos
    while inicio < hoy:
        fin = inicio + timedelta(days=182)
        if fin > hoy:
            fin = hoy
        intervalo_str = (
            f"{inicio.isoformat()}T00:00:00UTC",
            f"{fin.isoformat()}T00:00:00UTC"
        )
        intervalos.append(intervalo_str)
        inicio = fin + timedelta(days=1)

    # 3) Observo si ya existe un CSV para no bajar todo de nuevo
    ARCHIVO_SALIDA = "data/temperaturas_historicas_todas.csv"
    ya_bajadas = set()
    if os.path.exists(ARCHIVO_SALIDA):
        try:
            df_prev = pd.read_csv(ARCHIVO_SALIDA, dtype=str, usecols=["idema"])
            ya_bajadas = set(df_prev["idema"].dropna().unique())
        except Exception:
            ya_bajadas = set()

    # 4) Se crea un ID para esta ronda de descarga
    id_descarga = str(uuid.uuid4())
    print(f"ID de esta bajada: {id_descarga}")
    print(f"Hay {len(estaciones_df)} estaciones para procesar.")

    # 5) Bajamos datos por estación
    todos_registros = []
    for i, fila in estaciones_df.iterrows():
        indicativo = fila["indicativo"]
        nombre = fila.get("nombre", "")

        if indicativo in ya_bajadas:
            print(f"Ya bajé {indicativo}, paso al siguiente.")
            continue

        print(f"Bajando datos de {indicativo} – {nombre} ({i+1}/{len(estaciones_df)})")
        reg = descargar_estacion(indicativo, nombre, id_descarga, intervalos)
        todos_registros.extend(reg)

    # 6) Y guardamoss todo en un CSV
    if todos_registros:
        df_final = pd.DataFrame(todos_registros)
        os.makedirs("data", exist_ok=True)
        if os.path.exists(ARCHIVO_SALIDA):
            df_final.to_csv(ARCHIVO_SALIDA, mode="a", index=False, header=False)
        else:
            df_final.to_csv(ARCHIVO_SALIDA, index=False)
        print(f"Listo, guardé '{ARCHIVO_SALIDA}' con {len(df_final)} filas nuevas.")
    else:
        print("Nada nuevo que bajar en esta ejecución.")

if __name__ == "__main__":
    main()

